# Chat with vision models


## Authenticate to GitHub Models


In [ ]:
import os

import openai
from dotenv import load_dotenv

load_dotenv(".env", override=True)

openai_client = openai.OpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.inference.ai.azure.com")

## Send an image by Data URI



In [ ]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

In [26]:
response = openai_client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    temperature=0.2,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "does my 5 year old need to be afraid of this insect in our bathroom?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("bathroom_insect.jpeg")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

The insect in the image is a crane fly, which is often mistaken for a giant mosquito due to its similar appearance. However, crane flies are harmless to humans and do not bite or sting. They are actually beneficial insects that help to pollinate plants and serve as a food source for other animals.

Therefore, your 5-year-old does not need to be afraid of this insect in your bathroom. You can simply explain to them that it's a crane fly and that it won't hurt them. If you're concerned about the presence of crane flies in your bathroom, you may want to check for any standing water or moisture issues that could be attracting them.


​

​
​

​

​
​​
​
​
​
​
​
​

​

​
​
​
​
​
#### Automated image captioning

In [27]:
response = openai_client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    temperature=0.5,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "Reply with an alt text for this image that is between 1 to 3 sentences. Do not reply with your reasoning.", "type": "text"},
                {"image_url": {"url": open_image_as_base64("azure_arch.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

A diagram showing the components of a chat application and their connections, including Container App, Managed Identity, and Azure AI services. The Container App is linked to Managed Identity and Container Apps Environment. The components are represented by icons and labels.


​

​
​

​

​
​​
​
​
​
​
​
​

​

​
​
​
​
​

#### Assistance for vision-impaired

In [28]:
response = openai_client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "is there anything good for vegans on this menu?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("menu.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

**Vegan Options on the Menu**

Upon examining the menu, several options can be considered vegan-friendly with some modifications. The following dishes are potentially suitable for a vegan diet:

*   **Spinaci Soffritti**: Fresh spinach sautéed with lemon and garlic. This dish is naturally vegan.
*   **Panzanella con Fagioli**: Vine tomatoes and bread salad with sliced onions, basil, marinated Cannellini beans, cucumbers, and avocado. To make it vegan, request that the grilled chicken or steak be omitted.
*   **Insalata Di Mista**: Seasonal greens tossed with Carmine's house vinaigrette. This salad is vegan-friendly; however, be aware that it can be customized with grilled chicken or steak, which should be avoided.
*   **Zuppa Di Cavolo**: Cream of cauliflower Gratinée. This soup is likely to be vegan, but it's essential to confirm the ingredients used in the cream.
*   **Zuppa Di Fagioli**: Tuscan Beans and Pasta. This soup appears to be vegan, but verification of the broth and any add

​

​
​

​

​
​​
​
​
​
​
​
​

​

​
​
​
​
​


#### Insurance claim processing

In [29]:
response = openai_client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    messages=[
        {
            "role": "system",
            "content": (
                "You are an AI assistant that helps auto insurance companies process claims."
                "You accept images of damaged cars that are submitted with claims, and you are able to make judgments "
                "about the causes of automobile damage, and the validity of claims regarding that damage."
            ),
        },
        {
            "role": "user",
            "content": [
                {"text": "Claim states that this damage is due to hail. Is it valid?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("dented_car.jpg")}, "type": "image_url"},
            ],
        },
    ],
)

print(response.choices[0].message.content)

The claim that the damage is due to hail is not valid. The damage appears to be a large dent on the hood of the vehicle, which is not consistent with typical hail damage. Hail damage usually results in multiple small dents across the surface of the vehicle, rather than a single large dent. The damage shown in the image is more likely caused by a collision or other impact.


​

​
​

​

​
​​
​
​
​
​
​
​

​

​
​
​
​
​

#### Graph analysis

In [30]:
response = openai_client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "What zone are we losing the most trees in?", "type": "text"},
                {
                    "image_url": {
                        "url": open_image_as_base64("graph.png"),
                    },
                    "type": "image_url",
                },
            ],
        }
    ],
    temperature=0.5,
)

print(response.choices[0].message.content)

The chart shows the global tree cover annual loss in different climate zones from 2001 to 2023. The four climate zones are Boreal, Temperate, Subtropical, and Tropical.

To determine which zone is losing the most trees, we need to look at the height of the bars for each zone. The Tropical zone has the tallest bars, indicating the largest annual loss of tree cover.

**Key Observations:**

*   The Tropical zone consistently has the largest annual loss of tree cover across all years.
*   The annual loss in the Tropical zone is significantly higher than in the other three zones.
*   The chart provides a visual representation of the data, making it clear that the Tropical zone is experiencing the most significant loss of tree cover.

**Conclusion:**

Based on the chart, the Tropical zone is losing the most trees.


​

​
​

​

​
​​
​
​
​
​
​
​

​

​
​
​
​
​

#### Table analysis

In [31]:
response = openai_client.chat.completions.create(
    model="Llama-4-Maverick-17B-128E-Instruct-FP8",
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "What's the cheapest plant?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("page_0.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

To determine the cheapest plant, we need to examine the prices listed in the table.

**Step 1: Identify the Relevant Column**
The relevant column for this task is the "Price" column.

**Step 2: Scan the Price Column**
Upon scanning the "Price" column, we find various prices associated with different plants.

**Step 3: Compare Prices**
Comparing the prices:

*   $0.58 (Agrostis pallens - Thinggrass)
*   $1.15 (Bouteloua gracilis - Blue grama, StubS)
*   $1.83 (Centromadia pungens - Common tarweed, 4"5; Madia elegans - Tarweed, 4"5)
*   $2.30 (Anthoxanthum occidentale - Vanilla grass, Stub)
*   $3.60 (Eschscholzia caespitosa - Tufted Poppy, D-16S; Eschscholzia californica - California poppy, D-16S; Eschscholzia californica var. maritima - Coastal California Poppy, D-16S)
*   $3.65 (Epilobium densiflorum - Dense Spike-primrose, 4"5; Mentzelia lindleyi - Lindley's Blazing Star, 3"5)
*   $5.40 (Symphyotrichum subulatum - Slim marsh aster, D-16S; Trichostema lanceolatum - Vinegar weed, D-16S